In [9]:
# !pip install polars
# !pip install vaex
# !pip install duckdb
# !pip install fastparquet

In [10]:
from pathlib import Path

import pandas as pd
import vaex as vx
import polars as pl
import duckdb
import numpy as np

from IPython.display import Markdown, display

In [11]:
def printmd(string):
    display(Markdown(string))

In [12]:
# N = 1000000
# df = pd.DataFrame(np.random.standard_normal((N,100)))
# # Parquet requires string column names
# df.columns = [str(el) for el in df.columns]
# rawDataPath = Path('../data/raw')
# csvFile = rawDataPath / 'large.csv'
# parquetFile = rawDataPath / 'large.parquet'
# print('Write CSV')
# df.to_csv(csvFile,index=False)
# print('Write Parquet')
# df.to_parquet(parquetFile,index=False)
                  
    

### Compare size of the files on disk

#### Parquet is much smaller even though this dataset with lots of floats is not highly compressible

In [13]:
!du -h ../data/raw/large.*

1.8G	../data/raw/large.csv
805M	../data/raw/large.parquet


The time benchmarks here should be taken with a pinch of salt. These libraries have many distinct features beyond that captured here. For example:
- DuckDB is a very lightweight dependancy that makes it highly portable - for example it could be deployed on an instrument
- DuckDB has aimed for a wide testing base rather than optimising for performance
- Vaex is optimised for large (>100GB) datasets
- etc

In [14]:
# to start an in-memory database for duckdb
con = duckdb.connect(database=':memory:')

# Compare read time for a CSV

In [15]:
printmd("**Pandas with default C parser**")
%timeit -n 1 -r 1 pd.read_csv(csvFile)

**Pandas with default C parser**

24.8 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [16]:
printmd("**Pandas with pyarrow parser**")
%timeit -n 1 -r 1 pd.read_csv(csvFile,engine='pyarrow')

**Pandas with pyarrow parser**

6.61 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [17]:
printmd("**Vaex**")
%timeit -n 1 -r 1 vx.read_csv(csvFile)

**Vaex**

29.9 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [18]:
printmd("**Polars**")
%timeit -n 1 -r 1 pl.read_csv(csvFile)

**Polars**

3.16 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [19]:
printmd("**DuckDB**")
%timeit -n 1 -r 1 con.execute(f"SELECT * FROM '{csvFile}';").fetchdf()

**DuckDB**

24.1 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


# Compare read time for a Parquet file

In [20]:
printmd("**Pandas with default parquet reader**")
%timeit -n 1 -r 1 pd.read_parquet(parquetFile)

**Pandas with default parquet reader**

4.61 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [21]:
printmd("**Pandas with pyarrow parquet reader**")
%timeit -n 1 -r 1 pd.read_parquet(parquetFile,engine='pyarrow')

**Pandas with pyarrow parquet reader**

1.74 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [22]:
printmd("**Pandas with fastparquet reader**")
%timeit -n 1 -r 1 pd.read_parquet(parquetFile,engine='fastparquet')

**Pandas with fastparquet reader**

1.7 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [23]:
printmd("**Vaex**")
%timeit -n 1 -r 1 vx.open(parquetFile)

**Vaex**

3.01 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [24]:
printmd("**Polars**")
%timeit -n 1 -r 3 pl.read_parquet(parquetFile)

**Polars**

557 ms ± 43.9 ms per loop (mean ± std. dev. of 3 runs, 1 loop each)


In [25]:
printmd("**Polars with pyarrow**")
%timeit -n 1 -r 3 pl.read_parquet(parquetFile,use_pyarrow=True)

**Polars with pyarrow**

1.12 s ± 32.6 ms per loop (mean ± std. dev. of 3 runs, 1 loop each)


In [26]:
printmd("**DuckDB**")
%timeit -n 1 -r 1 con.execute(f"SELECT * FROM '{parquetFile}';").fetchdf()

**DuckDB**

3.03 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [28]:
def groupbyPandas(df:pd.DataFrame):
    return df.groupby(df.columns[0]).mean()
df = pl.read_parquet(parquetFile).to_pandas()
gbPandas = groupbyPandas(df=df)


In [30]:
def groupbyPolars(df:pl.DataFrame):
    gb = df.lazy().groupby('0').agg([pl.mean(df.columns[1:])]).sort("0", reverse=False)
    return gb.collect()
df = pl.read_parquet(parquetFile)
gbPolars = groupbyPolars(df=df)
pd.testing.assert_frame_equal(gbPandas,gbPolars.to_pandas().set_index('0'))

In [ ]:
df.head()

## Simple groupby operation

In [31]:
df = pl.read_parquet(parquetFile).to_pandas()
printmd('**Pandas**')
%timeit -n 1 -r 1 groupbyPandas(df=df)

**Pandas**

11.1 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [32]:
df = pl.read_parquet(parquetFile)
printmd('**Polars**')
%timeit -n 1 -r 1 groupbyPolars(df=df)

**Polars**

3.2 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
